# PI Controller
This is an example that shows how to use the PI controller to control a BHP by adjusting a choke opening.
It will first set a constant flow rate and choke opening and then after 60 seconds the BHP will be controlled by varying the choke opening (via a PI controller) while maintaining a constant flow rate
Can be used to show the importance of proper PID tuning

In [ ]:
import openlab
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Input python generated scripts from live.openlab.app
username = None
apikey = None
licenseguid = None

In [ ]:
# login and setup client
session = openlab.http_client(username=username, apikey=apikey, licenseguid=licenseguid)

In [ ]:
config_name = "test"
sim_name = "Simulation from Notebook"
initial_depth = 2500

## PID Tuning

In [ ]:
#PI controller settings
referenceBHPPressure = 380 *100000 # Pa
initialChokeOpening = 0.25 # 0 = closed , 1 = open

kp = -0.02 # first try for tuning the pi controller
ki = kp/10
ts = 1
pi = openlab.piController.Controler(kp, ki, ts, referenceBHPPressure,initialChokeOpening)

In [ ]:
# results we want 
tags=["SPP", "DownholeECD", "FlowRateOut", "ChokeOpening", "DownholePressure"]

In [ ]:
#initialize simulation
sim = session.create_simulation(config_name,sim_name,2500)
timeStep = 1

In [ ]:
#units
FLOW_UNIT_CONV_FACTOR= 1.66666667 * 0.00001#float("10e-5") # l/min --> m^3/s
PRESSURE_CONV_FACTOR= 100000.0 # float("10e5") # bar-->pascal

In [ ]:
#Plot stuff
%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## Running the simulation

In [ ]:
#configure plot 
fig, ((plot_q, plot_choke), (plot_bhp,plot_spp)) = plt.subplots(2, 2,sharex=True)

startTime = timeStep
endTime = startTime + 150

plot_q.set_xlim(startTime,endTime)
plot_bhp.set_xlim(startTime,endTime)
plot_q.set_ylim(1500,3000) #3000 l/min
plot_choke.set_ylim(0,0.5)   #fraction
plot_bhp.set_ylim(300,400) #500 bar
plot_spp.set_ylim(0,500) #500 bar

plot_q.set_title('Flow Rate Out (l/min)')
plot_choke.set_title('Choke Opening')
plot_bhp.set_title('Downhole Pressure(bar)')
plot_spp.set_title('SPP (bar)')
plot_bhp.set_xlabel('Time Step (seconds)')
plot_spp.set_xlabel('Time Step (seconds)')


plt.ion() #make plot interactive
plt.show()
fig.canvas.draw()

#empty lists for plots
steps_ = list()
spps_ = list()
qs_= list()
chokes_= list()
ps_ = list()

rampStartTime = timeStep + 60
initialFlowRate = 2000/60000
targetFlowRate = 2500/60000

for timeStep in range(startTime,endTime):
    
    if timeStep >= rampStartTime:
        if timeStep == rampStartTime:
            #reset PI controller before usage, set reference value and initial output = initial choke opening
            pi.reset(referenceBHPPressure, sim.results.ChokeOpening[timeStep-1])
            flowRateIn = targetFlowRate 
        
        chokeOpening = pi.getOutput(sim.results.DownholePressure[timeStep-1])

    else: #constant flow rate and choke opening
        flowRateIn = 2000/60000
        chokeOpening = initialChokeOpening

    #set setpoints 
    sim.setpoints.FlowRateIn = flowRateIn
    sim.setpoints.ChokeOpening = chokeOpening
    
    #step simulator
    sim.step(timeStep)         

    #ask results
    sim.get_results(timeStep,tags)

    spp = sim.results.SPP[timeStep]/PRESSURE_CONV_FACTOR
    p = sim.results.DownholePressure[timeStep]/PRESSURE_CONV_FACTOR
    q = sim.results.FlowRateOut[timeStep]/FLOW_UNIT_CONV_FACTOR
    choke = sim.results.ChokeOpening[timeStep]
    
    plot_q.scatter(timeStep , q, c='b', s= 1) #c = color; s = size
    plot_choke.scatter(timeStep,choke, c='b', s = 1)
    plot_bhp.scatter(timeStep,p, c='b',s=1)
    plot_spp.scatter(timeStep,spp, c='b',s=1)
    
    fig.canvas.draw()   # draw
    
    #advance the simulation    
    timeStep = timeStep + 1

*Note that after the above for loop has completed, the graph can be dragged to make it bigger

We see that the controller is not tuned correctly because there are large oscillations, meaning that the kp gain is too high.

Common PID tuning strategy is to reduce the kp value by half until no oscillations are present.

Go back to the "PID Tuning" section above, and change kp from -0.02 to -0.01 and hit run to reset the controller.

Now run the simulation again by pressing run and notice how much the oscillations have imporved

In [ ]:
#stop the sim once you have experimenting some with tuning the PI controller
sim.stop()